In [1]:
!pip install dotenv

In [2]:
from dotenv import load_dotenv
import requests
from os import getenv
from urllib.parse import quote
import pandas as pd
from IPython.display import HTML, display
import re
from os import path

In [7]:
load_dotenv()
DM_USER = getenv("DM_USER")
DM_PASS = getenv("DM_PASS")
browser_url = "https://prelude.dotmatics.net/browser/api"

token_endpoint = f"{browser_url}/authenticate/requestToken?expiration=5000"
exp_id_list = [
    138606,
    140442,
    143526,
    143545,
    145301,
    146623,
    146696,
    147096,
    149864,
    150324,
    152823,
    153056,
    156573,
    157390,
    157845,
    157858,
    157884,
    157885,
    157887,
    158504,
    158648,
    158804,
    159305,
    159568,
    159826,
    160317,
    160324,
    160325,
    160326,
    160826,
    160944,
    160949,
    161059,
    161352,
    161635,
    163275,
    165404,
    168950,
    170145,
    176069,
    176683,
    176775,
    178004,
    178326,
    180354,
    180525,
    180636,
    180665,
    180666,
    180682,
    180791,
    180892,
    180936,
    183092,
    183194,
    183464,
    183767,
    183850,
    183975,
    184197,
    184623,
    184657,
    185889,
    190882,
    190883,
    191125,
    191434,
    191865,
    192261,
    192580,
    192929,
    193350,
    196064,
    196086,
    196087,
    196329,
    196355,
    196423,
    196586,
    197164,
    197791,
    197915,
    198490,
    199296,
    199477,
    199908,
    199923,
    200146,
    200639,
    201526,
    201767,
    202113,
    202578,
    202638,
    202678,
    202681,
    203497,
    203535,
    205469,
    205473,
    205931,
    206064,
    206108,
    206117,
    206118,
    206121,
    206441,
    206594,
    207085,
    207168,
    207754,
    207912,
    207920,
    210275,
    210473,
    210847,
    216848,
    217168,
    217247,
    220381,
    225434,
    225700,
    226053,
    226247,
    226306,
    226343,
    226407,
    227022,
    229371,
    230117,
    230797,
    230840,
    231094,
    231468,
    231614,
    231827,
    231858,
    232399,
    232587,
    232996,
    233004,
    233101,
    233706,
    234064,
    234862,
    235011,
    235176,
    235916,
    236148,
    236533,
    239297,
    239380,
    240666,
    240787,
    240885,
    240888,
    242757,
    243064,
    245935,
    246609,
    247426,
    247948,
    251519,
    251915,
    251916,
    251917,
    251918,
    251919,
    251920,
    252772,
    253050,
    253374,
    254023,
    254282,
    255133,
    256068,
    256451,
    256511,
    256522,
    256875,
    256948,
    257084,
    257177,
    257443,
    258356,
    259087,
    259383,
    259417,
    259421,
    259422,
    259463,
    259668,
    260178,
    260345,
    260516,
    260570,
    260667,
    261401,
    261819,
    262058,
    265928,
    266388,
    267059,
    267088,
    267198,
    267203,
    267217,
    267342,
    267397,
]


def fetch_writeups(exp_id_list, writeup_filename):
    token = requests.get(token_endpoint, auth=(DM_USER, DM_PASS)).json()
    headers = {"Authorization": f"Dotmatics {token}"}
    writeups_data = []
    
    for i, exp_id in enumerate(exp_id_list):
        try:
            writeup_url = f"{browser_url}/studies/experiment/{exp_id}/writeup/{{includeHtml}}"
            response = requests.get(writeup_url, headers=headers)
            
            data = {
                "index": i,
                "exp_id": exp_id,
                "writeup": response.json(),
                "url": writeup_url
            }
            
            writeups_data.append(data)
            print(data)
        except Exception as e:
            print(f"Error fetching {exp_id}: {str(e)}")
    
    df = pd.DataFrame(writeups_data)
    df.to_pickle(writeup_filename)

    return df


def display_full_width(content):
    display(HTML(
        f"<div style='margin: 10px 0; width: 100%; display: flex; "
        f"border: 1px solid #ddd; border-radius: 4px; background: #f8f9fa;'>"
        f"{content}</div>"
    ))


def highlight_question_marks(text):
    return re.sub(r'\?', "<span style='background-color: yellow; color: black;'>?</span>", text)


filtered_df_pkl = 'writeups_filtered_df.pkl'
if not path.isfile(filtered_df_pkl):
    fetch_writeups(exp_id_list, filtered_df_pkl)
else:
    df = pd.read_pickle(filtered_df_pkl)

In [28]:
df.head()

,index,exp_id,writeup,url
0,0,138606,"A mixture of ? (100.mg, 0.3500mmol) sodium io...",https://prelude.dotmatics.net/browser/api/stud...
1,1,140442,A 20 mL vial containing tert-butyl N-[(Z)-[7-[...,https://prelude.dotmatics.net/browser/api/stud...
2,2,143526,"To (R)-[(3aS,4R,6R,6aR)-4-(4-chloropyrrolo[2,3...",https://prelude.dotmatics.net/browser/api/stud...
3,3,143545,"To (R)-[(3aS,4R,6R,6aR)-4-(4-chloropyrrolo[2,3...",https://prelude.dotmatics.net/browser/api/stud...
4,4,145301,"To a solution of 2,2,2-trichloroethane-1,1-di...",https://prelude.dotmatics.net/browser/api/stud...


In [5]:
for _, row in df.iterrows():
    highlighted_writeup = highlight_question_marks(str(row['writeup']))
    
    html = f"""
    <div style="display: flex; flex-direction: column; width: 100%; border: 2px solid #bbb;">
        <!-- Header Row -->
        <div style="display: flex; font-weight: bold; background: #d6d8db; padding: 8px; border-bottom: 2px solid #bbb;">
            <div style="width: 80px; text-align: center; border-right: 2px solid #bbb; display: flex; align-items: center; justify-content: center;">Index</div>
            <div style="width: 150px; text-align: center; border-right: 2px solid #bbb; display: flex; align-items: center; justify-content: center;">Experiment ID</div>
            <div style="flex: 1; text-align: center;">Writeup</div>
        </div>

        <!-- Data Row -->
        <div style="display: flex; padding: 8px; background: #f8f9fa;">
            <div style="width: 80px; text-align: center; border-right: 2px solid #ddd; font-weight: bold; display: flex; align-items: center; justify-content: center;">
                {row['index']+1}
            </div>
            <div style="width: 150px; text-align: center; border-right: 2px solid #ddd; font-weight: bold; display: flex; align-items: center; justify-content: center;">
                {row['exp_id']}
            </div>
            <div style="flex: 1; padding: 8px 12px; white-space: pre-wrap; overflow-wrap: break-word;">
                {highlighted_writeup}
            </div>
        </div>
    </div>
    """

    display_full_width(html)

In [ ]:
exp_id_list_diff = [
    138784,
    139184,
    139229,
    139333,
    139366,
    139547,
    139559,
    139673,
    139686,
    139710,
    139716,
    139971,
    140199,
    141394,
    141569,
    141588,
    141851,
    142022,
    142248,
    142622,
    142648,
    142649,
    142709,
    142710,
    142726,
    142745,
    142746,
    142763,
    142861,
    142863,
    143109,
    143325,
    143524,
    143794,
    144419,
    144441,
    144485,
    144488,
    144562,
    144584,
    144586,
    144704,
    144832,
    144957,
    145150,
    145431,
    145489,
    145637,
    145718,
    145790,
    145866,
    146013,
    147516,
    147614,
    148010,
    148351,
    148628,
    148968,
    149057,
    149658,
    149828,
    149939,
    150441,
    150476,
    150560,
    152548,
    152647,
    152856,
    152858,
    152863,
    153261,
    153430,
    153633,
    153986,
    154642,
    155407,
    155718,
    156157,
    156771,
    156831,
    157251,
    158461,
    158654,
    158660,
    161500,
    162024,
    162250,
    162374,
    163457,
    164157,
    164445,
    164849,
    164975,
    164978,
    165177,
    165617,
    165619,
    165621,
    166428,
    167183,
    168421,
    168750,
    169020,
    169928,
    169978,
    170055,
    170058,
    170101,
    170341,
    171819,
    173148,
    173425,
    173788,
    174005,
    174093,
    174595,
    175917,
    176519,
    178168,
    178236,
    178509,
    179448,
    179522,
    179528,
    179540,
    181336,
    181414,
    182372,
    182408,
    182409,
    182421,
    182474,
    182582,
    183453,
    184218,
    184431,
    184850,
    184929,
    185023,
    185050,
    185178,
    185442,
    185851,
    185976,
    185983,
    186945,
    187173,
    187241,
    187549,
    188768,
    189414,
    189706,
    189811,
    190835,
    191075,
    191111,
    191121,
    191126,
    191364,
    192075,
    192138,
    192373,
    192526,
    193090,
    193428,
    193540,
    193860,
    193886,
    194049,
    194312,
    194501,
    194671,
    194839,
    195369,
    195459,
    195662,
    195863,
    196395,
    196717,
    196852,
    197285,
    197305,
    197446,
    198159,
    198707,
    199342,
    200121,
    200278,
    200546,
    200547,
    201388,
    202774,
    205491,
    206192,
    206671,
    206798,
    206834,
    206881,
    206890,
    207072,
    207073,
    207075,
    207174,
    207175,
    207312,
    207321,
    207393,
    207414,
    207509,
    207532,
    207577,
    207610,
    208248,
    208277,
    208459,
    208467,
    208553,
    208674,
    208745,
    209046,
    209098,
    209101,
    209159,
    209188,
    209224,
    209252,
    209272,
    209356,
    209495,
    209566,
    209567,
    209584,
    209605,
    209609,
    209650,
    209670,
    209674,
    209701,
    209910,
    209924,
    210012,
    210587,
    211189,
    211323,
    212460,
    213067,
    213635,
    213638,
    214150,
    214151,
    214512,
    214829,
    214963,
    215385,
    215912,
    215978,
    217359,
    218368,
    218952,
    219167,
    219344,
    220449,
    220712,
    221389,
    221409,
    222169,
    222171,
    222311,
    222568,
    222577,
    222641,
    223370,
    223469,
    223941,
    224488,
    225006,
    225047,
    225153,
    225250,
    225428,
    225432,
    225909,
    226664,
    226713,
    227266,
    227915,
    228197,
    228299,
    228849,
    228941,
    229012,
    229273,
    229274,
    229499,
    229828,
    230790,
    230801,
    231377,
    231387,
    231391,
    232096,
    232209,
    232210,
    232613,
    232664,
    233076,
    233253,
    233542,
    233543,
    233555,
    234146,
    234273,
    234279,
    234290,
    234295,
    234615,
    235222,
    235516,
    235612,
    235619,
    236443,
    236444,
    236445,
    237070,
    237135,
    237324,
    237418,
    237943,
    238664,
    238692,
    238710,
    238969,
    239156,
    239180,
    239250,
    239328,
    239334,
    239469,
    239661,
    239752,
    239769,
    240216,
    240372,
    241073,
    241293,
    241304,
    241316,
    241470,
    241520,
    242037,
    242195,
    242203,
    242376,
    242566,
    244062,
    244353,
    244442,
    244803,
    245036,
    245109,
    245281,
    245406,
    245421,
    245689,
    245923,
    246226,
    246235,
    246236,
    246240,
    246321,
    246452,
    246896,
    247328,
    247462,
    247466,
    247552,
    247596,
    247766,
    247883,
    247920,
    248077,
    248188,
    248570,
    248615,
    248663,
    248728,
    249031,
    249562,
    249646,
    249648,
    249667,
    249682,
    249683,
    249983,
    250074,
    250111,
    250427,
    250757,
    251509,
    251603,
    252200,
    252260,
    252426,
    252435,
    252681,
    252950,
    253068,
    253070,
    253083,
    253253,
    253267,
    253421,
    253567,
    253945,
    254040,
    254355,
    254643,
    254692,
    254849,
    255230,
    255428,
    255995,
    256108,
    256109,
    256116,
    256200,
    256275,
    256350,
    256380,
    256663,
    256665,
    256685,
    256872,
    257159,
    257344,
    257540,
    257564,
    257903,
    257974,
    258523,
    258664,
    258794,
    260163,
    260172,
    260245,
    260288,
    260500,
    261177,
    261289,
    261317,
    261613,
    261887,
    261968,
    261991,
    262117,
    262732,
    262912,
    263216,
    263319,
    263405,
    263621,
    263836,
    264030,
    264979,
    265046,
    265048,
    265050,
    265162,
    265296,
    265488,
    265489,
    265743,
    265784,
    266135,
    266494,
    266559,
    266666,
    267076,
    267101,
    267383
]

all_df_pkl = 'writeups_all_df.pkl'
if not path.isfile(all_df_pkl):
    fetch_writeups(exp_id_list_diff, all_df_pkl)
else:
    df = pd.read_pickle(all_df_pkl)

{'index': 0, 'exp_id': 138784, 'writeup': 'Purchased from Cayman to be tested in PRMT assays Item:13603, batch 0459019-10 http://www.commonchemistry.org/ChemicalDetail.aspx?ref=979-92-0 ', 'url': 'https://prelude.dotmatics.net/browser/api/studies/experiment/138784/writeup/{includeHtml}'}
{'index': 1, 'exp_id': 139184, 'writeup': 'Standard Controls Apply: Lab coats, gloves and safety specs are to be worn at all times. All experiments should be performed in a fumehood.  Pfizer patent Ex. 92/93 Step I: Synthesis of 7-((3aR,4R,6S,6aS)-6-((3,4- difluorophenyl)fluoromethyl)-2,2-dimethyltetrahydrofuro[ 3,4-d] [1,3]dioxol-4-yl)-4-methyl-7H-pyrrolo [2,3-d]pyrimidine (RR-I) SS-I [0460] To a solution of SS-I (Scheme SS) (500 mg, 1.20 mmol) in 5 mL CH2Cl2 was added DAST (965 mg, 5.99 mmol), stirred at rt for 30 min. The reaction was quenched with water, transferred to a separatory funnel, and diluted with 10 mL water. The phases were separated and the aqueous phase was extracted with 3 portions of

In [ ]:
for _, row in df.iterrows():
    highlighted_writeup = highlight_question_marks(str(row['writeup']))
    
    html = f"""
    <div style="display: flex; flex-direction: column; width: 100%; border: 2px solid #bbb;">
        <!-- Header Row -->
        <div style="display: flex; font-weight: bold; background: #d6d8db; padding: 8px; border-bottom: 2px solid #bbb;">
            <div style="width: 80px; text-align: center; border-right: 2px solid #bbb; display: flex; align-items: center; justify-content: center;">Index</div>
            <div style="width: 150px; text-align: center; border-right: 2px solid #bbb; display: flex; align-items: center; justify-content: center;">Experiment ID</div>
            <div style="flex: 1; text-align: center;">Writeup</div>
        </div>

        <!-- Data Row -->
        <div style="display: flex; padding: 8px; background: #f8f9fa;">
            <div style="width: 80px; text-align: center; border-right: 2px solid #ddd; font-weight: bold; display: flex; align-items: center; justify-content: center;">
                {row['index']+1}
            </div>
            <div style="width: 150px; text-align: center; border-right: 2px solid #ddd; font-weight: bold; display: flex; align-items: center; justify-content: center;">
                {row['exp_id']}
            </div>
            <div style="flex: 1; padding: 8px 12px; white-space: pre-wrap; overflow-wrap: break-word;">
                {highlighted_writeup}
            </div>
        </div>
    </div>
    """

    display_full_width(html)